In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Langsmith tracking
os.environ["LANGSMITH_TRACKING"] = "true"
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x0000015CD420FB60> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000015CD42416D0> root_client=<openai.OpenAI object at 0x0000015CD40C3E60> root_async_client=<openai.AsyncOpenAI object at 0x0000015CD420FBC0> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


prompt 1

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context.
Think step by step before providing a detailed answer.
I will tip you $100000 if the user finds the answer helpful.
<context>
{context}
</context>                                                                                                                                                                
Question: {input}
""")

prompt 2

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt_2=ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert Data Scientist and Gen AI Engineer. Provide me answers based on the asked question "),
        ("user","{input}")

    ]
)
prompt_2

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert Data Scientist and Gen AI Engineer. Provide me answers based on the asked question '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

## Chain

In [6]:
from langchain_core.output_parsers import StrOutputParser
output_parser=StrOutputParser()
chain=prompt_2|llm|output_parser

response=chain.invoke({"input":"Can you tell me about Langsmith?"})
print(response)

Langsmith is a platform associated with LangChain, a framework designed for building applications using large language models. Langsmith serves as a suite of tools that empower developers to effectively prototype, evaluate, and deploy applications that leverage LLMs (Large Language Models) and their associated chains. The platform includes features for observing model performance, testing application outputs, and debugging issues in real-time, facilitating a smooth development process from the initial stages to production. Langsmith aims to streamline the workflow for developers working on language model-based applications, enabling more efficient and effective use of LLMs in various applications.


In [7]:
'''from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
output_parser=JsonOutputParser()
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)'''

'from langchain_core.prompts import PromptTemplate\nfrom langchain_core.output_parsers import JsonOutputParser\noutput_parser=JsonOutputParser()\nprompt = PromptTemplate(\n    template="Answer the user query.\n{format_instructions}\n{query}\n",\n    input_variables=["query"],\n    partial_variables={"format_instructions": output_parser.get_format_instructions()},\n)'

In [8]:
'''chain = prompt_2|llm|output_parser

response=chain.invoke({"query":"Can you tell me about Langsmith?"})
print(response)'''

'chain = prompt_2|llm|output_parser\n\nresponse=chain.invoke({"query":"Can you tell me about Langsmith?"})\nprint(response)'

home_work = output parser, langchain python doc

## Document Chain

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

## Data Ingestion

In [10]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader("https://www.credo.ai/blog/cutting-through-the-noise-what-is-ai-governance", bs_kwargs={
        "parse_only": bs4.SoupStrainer(class_="text-rich-text w-richtext"),
    }
)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [11]:
doc = loader.load()
document = doc[0].page_content

## Splitting

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(document)

## Embedding

In [13]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

## Vector store

In [14]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_texts(texts=texts, embedding=embedding)

## Retriever

In [15]:
retriever = db.as_retriever(search_type="mmr")

## Retriever Chain

### User Query

In [16]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [17]:
response = retrieval_chain.invoke({"input": "what is Ai Governance?"})

In [18]:
response['answer']

'AI Governance is a framework of policies and processes that guide the development, deployment, use, and management of AI systems. Its aim is to maximize the benefits of AI technologies while mitigating their risks. To achieve this, AI Governance involves the collaboration of diverse stakeholders— such as AI researchers, engineers, companies, and regulatory bodies—working together to ensure ethical considerations and safety measures are integrated throughout the AI lifecycle.\n\nKey aspects of AI Governance include:\n\n1. **Policy and Oversight**: Establishing a set of guiding principles and rules to ensure AI systems operate safely and effectively, providing oversight to ensure adherence to these measures.\n\n2. **Risk Mitigation**: Identifying and addressing potential risks associated with AI systems to prevent harmful outcomes, similar to the way governance in other areas aims to mitigate risks.\n\n3. **Evaluation and Monitoring**: Continuously assessing AI systems using diverse met